<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/GPAW_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run GPAW on Google-colab

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


Install gpaw and other libraries from conda

In [2]:
!conda install -y -c conda-forge gpaw

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.



In [3]:
!conda install -y -c conda-forge openmpi=4.1.2 gcc

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.



In [4]:
!export GPAW_SETUP_PATH='/content'
!gpaw install-data .

Available setups and pseudopotentials
  [*] https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.20000.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.11271.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.9.9672.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.8.7929.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.6.6300.tar.gz
      https://wiki.fysik.dtu.dk/gpaw-files/gpaw-setups-0.5.3574.tar.gz

Selected gpaw-setups-0.9.20000.tar.gz.  Downloading...
Extracting tarball into .
Setups installed into /content/gpaw-setups-0.9.20000.
Register this setup path in /root/.gpaw/rc.py? [y/n] y
It looks like the path is already registered in /root/.gpaw/rc.py.
File will not be modified at this time.
Installation complete.


In [5]:
!gpaw test

 ------------------------------------------------------------------------------------------------
| python-3.7.10     /usr/local/bin/python                                                        |
| gpaw-22.1.0       /usr/local/lib/python3.7/site-packages/gpaw/                                 |
| ase-3.22.1        /usr/local/lib/python3.7/site-packages/ase/                                  |
| numpy-1.21.6      /usr/local/lib/python3.7/site-packages/numpy/                                |
| scipy-1.7.3       /usr/local/lib/python3.7/site-packages/scipy/                                |
| libxc-5.2.3       yes                                                                          |
| _gpaw             /usr/local/lib/python3.7/site-packages/_gpaw.cpython-37m-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                          |
| OpenMP enabled    yes                                                                          |
| scalapack

In [6]:
!gpaw info

 ------------------------------------------------------------------------------------------------
| python-3.7.10     /usr/local/bin/python                                                        |
| gpaw-22.1.0       /usr/local/lib/python3.7/site-packages/gpaw/                                 |
| ase-3.22.1        /usr/local/lib/python3.7/site-packages/ase/                                  |
| numpy-1.21.6      /usr/local/lib/python3.7/site-packages/numpy/                                |
| scipy-1.7.3       /usr/local/lib/python3.7/site-packages/scipy/                                |
| libxc-5.2.3       yes                                                                          |
| _gpaw             /usr/local/lib/python3.7/site-packages/_gpaw.cpython-37m-x86_64-linux-gnu.so |
| MPI enabled       yes                                                                          |
| OpenMP enabled    yes                                                                          |
| scalapack

Note: might have to restart runtime

In [7]:
from ase import Atoms
from ase.visualize import view
from gpaw import GPAW, PW

name = 'Al-fcc'
a = 4.05  # fcc lattice parameter
b = a / 2

bulk = Atoms('Al',
             cell=[[0, b, b],
                   [b, 0, b],
                   [b, b, 0]],
             pbc=True)

view(bulk)

k = 4
calc = GPAW(mode=PW(300),       # cutoff
            kpts=(k, k, k),     # k-points
            txt=name + '.txt')  # output file

bulk.calc = calc

energy = bulk.get_potential_energy()
calc.write(name + '.gpw')
print('Energy:', energy, 'eV')

Energy: -4.122151616077686 eV
